# The Wikipedia Game
The goal of this project is to build an algorithm that is able to perform better than a random player at the wikipedia game! The wikipedia game is [INSERT INFO ABOUT WIKIPEDIA GAME]. 

We've broken the project into our main technical challenges
* Scrape wikipedia (or at least a subset) in order to build a data structure with information about what links exist between pages.
* Use graph algorithms to learn cool things about our graph (including the shortest distance between two pages)!
* Create an algorithm that does not rely on the graph but can still navigate from one page to another in a short distance. Ideally this distance would be equal to the shortest possible distance, but it would be cool if it at least performed better than an algorithm that chooses links randomly.
* Compare algorithm to a human and random competitor

## Step 1: Scraping Wikipedia
The easiest way to do this would probably have been to rely on the wikipedia data dumps that are released twice monthly. However, we wanted to play with beautiful soup and requests, so we decided to implement the scraping manually. This is much more time intensive (computationally) which means that we are not able to build a graph of all of wikipedia, but we also found it to be way cooler!

Before we could actually scrape wikipedia, we had to decide what data we wanted to store and how we wanted to store it. After looking through a bunch of articles, we found that all wikipedia pages have the same link structure:
```
https://en.wikipedia.org/wiki/[articleTitle]
```
So the article on Star Wars has the link https://en.wikipedia.org/wiki/Star_Wars and the article entitled "The Hitchhiker's Guide to the Galaxy" has the link https://en.wikipedia.org/wiki/The_Hitchhiker%27s_Guide_to_the_Galaxy (notice how the apostraphe is handled!).

This meant that as long as we stored the article title, we could always recreate the link for the article.


In [114]:
import requests
from bs4 import BeautifulSoup     

PARSER = "lxml"

title_0 = r"The_Hitchhiker%27s_Guide_to_the_Galaxy"
wiki_url = "https://en.wikipedia.org/wiki/" + title_0
response = requests.get(wiki_url)
soup = BeautifulSoup(response.text,PARSER)

print(f"The title of the requested page is {soup.title}")

The title of the requested page is <title>The Hitchhiker's Guide to the Galaxy - Wikipedia</title>


Now the question is how do we find all of the article names linked to within a given article?

In [115]:
# First let's get the first 10 <a> html elements
a_elements = soup.find_all('a')[:10]

In [116]:
# Let's extract the href part of each <a> element
links = []
for element in a_elements:
    links.append(element.get('href'))

print(links)


[None, '/wiki/File:The_Hitchhikers_Guide_to_the_Galaxy_Part_1.ogg', '#mw-head', '#searchInput', '/wiki/The_Hitchhiker%27s_Guide_to_the_Galaxy_(disambiguation)', '/wiki/Hitchhiker%27s_Guide_(disambiguation)', '/wiki/File:H2G2_UK_front_cover.jpg', '/wiki/Douglas_Adams', '/wiki/The_Hitchhiker%27s_Guide_to_the_Galaxy_Primary_and_Secondary_Phases', '/wiki/The_Hitchhiker%27s_Guide_to_the_Galaxy:_The_Original_Radio_Scripts']


Clearly all of the links that start with "/wiki/" are wikipedia pages! So let's filter out everything else

In [117]:
clean_links = []
for link in links:
    if (link and link[:6] == "/wiki/"):
        clean_links.append(link)

links = clean_links.copy()
links

['/wiki/File:The_Hitchhikers_Guide_to_the_Galaxy_Part_1.ogg',
 '/wiki/The_Hitchhiker%27s_Guide_to_the_Galaxy_(disambiguation)',
 '/wiki/Hitchhiker%27s_Guide_(disambiguation)',
 '/wiki/File:H2G2_UK_front_cover.jpg',
 '/wiki/Douglas_Adams',
 '/wiki/The_Hitchhiker%27s_Guide_to_the_Galaxy_Primary_and_Secondary_Phases',
 '/wiki/The_Hitchhiker%27s_Guide_to_the_Galaxy:_The_Original_Radio_Scripts']

We probably also don't want the Files. Let's get rid of these two

In [118]:
clean_links = []
for link in links:

    # We found that any link with a : in it is not a "regular" wikipedia
    # Page so we exclude them all
    if (link and link.find(":") == -1):
        clean_links.append(link)

links = clean_links.copy()
links

['/wiki/The_Hitchhiker%27s_Guide_to_the_Galaxy_(disambiguation)',
 '/wiki/Hitchhiker%27s_Guide_(disambiguation)',
 '/wiki/Douglas_Adams',
 '/wiki/The_Hitchhiker%27s_Guide_to_the_Galaxy_Primary_and_Secondary_Phases']

We combined this scraping and filtering into our `get_wiki_graph_one_step` function. It takes a list of wikipedia article titles. It returns a dictionary where each article is a key and the corresponding value is a list of all the articles linked to within that article.

In [120]:
from scraping import get_wiki_graph_one_step, get_wiki_graph

title_1 = "Star_Wars"
d = get_wiki_graph_one_step([title_0, title_1])

print()
print(f"{title_0} links to {len(d[title_0])} other wikipedia articles")
print(f"For example, within the {title_0} article, there are links to the following wikipedia pages:")
for i in range(0,min(10, len(d[title_0]))):
    print(f"\t{d[title_0][i]}")

print()
print(f"{title_1} links to {len(d[title_1])} other wikipedia articles")
print(f"For example, within the {title_1} article, there are links to the following wikipedia pages:")
for i in range(0,min(10, len(d[title_1]))):
    print(f"\t{d[title_1][i]}")    

(0/2): The_Hitchhiker%27s_Guide_to_the_Galaxy
(1/2): Star_Wars

The_Hitchhiker%27s_Guide_to_the_Galaxy links to 298 other wikipedia articles
For example, within the The_Hitchhiker%27s_Guide_to_the_Galaxy article, there are links to the following wikipedia pages:
	Robbie_Stamp
	The_Hitchhiker%27s_Guide_to_the_Galaxy_(TV_series)
	Peter_Jones_(actor)
	La_Boite_Theatre_Company
	Hyperion_(publisher)
	Alan_J._W._Bell
	Extraterrestrial_life_in_popular_culture
	Zaphod_Beeblebrox
	Miriam_Margolyes
	Saeed_Jaffrey

Star_Wars links to 1258 other wikipedia articles
For example, within the Star_Wars article, there are links to the following wikipedia pages:
	Obi-Wan_Kenobi_(TV_series)
	Military_order_(religious_society)
	Luke_Skywalker_and_the_Shadows_of_Mindor
	Bo-Katan_Kryze
	Kathleen_Kennedy_(producer)
	The_Ewok_Adventure
	Action_Man_(1993%E2%80%932006_toyline)
	Avalon_Hill
	List_of_Star_Wars_Rebels_characters
	Star_Wars_Tales


Above, we just requested the star wars wikipedia page. But in order to build a bigger graph, we could then repeat the same process for all of the pages referenced in the star wars page. We could continue to repeat this process for as many steps as we'd like.

Let's try this:

In [3]:
title = "New_York_State_Route_373"
out = get_wiki_graph(starting_refs=[title], num_steps=2)

(0/1): New_York_State_Route_373
(0/59): The_New_York_Times
(1/59): Numbered_highways_in_New_York
(2/59): Lake_Champlain
(3/59): United_States
(4/59): 52nd_New_York_State_Legislature
(5/59): Vermont
(6/59): Parkways_in_New_York
(7/59): ISBN_(identifier)
(8/59): Main_Page
(9/59): Theodore_Roosevelt_International_Highway
(10/59): Ausable_Chasm
(11/59): American_Antiquarian_Society
(12/59): Canadian_Pacific_Railway
(13/59): New_York_State_Legislature
(14/59): Keeseville,_New_York
(15/59): Hamlet_(New_York)
(16/59): St._Lawrence_County,_New_York
(17/59): List_of_reference_routes_in_New_York
(18/59): Burlington,_Vermont
(19/59): New_York_State_Route_374
(20/59): Plattsburgh,_New_York
(21/59): General_Drafting
(22/59): Ausable_Chasm,_New_York
(23/59): U.S._Route_9_in_New_York
(24/59): County_Route_17_(Essex_County,_New_York)
(25/59): Standard_Oil_Company_of_New_York
(26/59): List_of_Interstate_Highways_in_New_York
(27/59): Burlington%E2%80%93Port_Kent_Ferry
(28/59): Portland,_Oregon
(29/59): 

In [11]:
d = out[0]
print(f"{title} has {len(d[title])} links")

title_1 = d[title][0]
print(f"One such link is {title_1} which itself has {len(d[title_1])} links all of which link out to other articles")


New_York_State_Route_373 has 60 links
One such link is The_New_York_Times which itself has 927 links all of which link out to other articles


Above we did this for two steps. In the first step we found all of the links in `New_York_State_Route_373`. In the second step we found all of the links within all of these links.

In [13]:
print(f"If we were to do the third step, we would have to find all of the links within {len(out[2])} wikipedia articles")

print(f"Given that this takes about 0.5 seconds per article, just three steps would take {len(out[2])*0.5} seconds or {len(out[2])*0.5/3600} hours")

If we were to do the third step, we would have to find all of the links within 19530 wikipedia articles
Given that this takes about 0.5 seconds per article, just three steps would take 9765.0 seconds or 2.7125 hours


With every additional step we take beyond the initial article, the time to build the graph increases exponentially. Since we didn't want our personal computers to be occupied for multiple days, we did this on the Pomona servers using the `scraping.py` file we wrote. The final product was a .json file. In the next section, we will analyze this json and use it to construct a graph.

## Step 2: Graph Algorithms
Let's first load in and take a look at the json we pulled in step 1.

In [122]:
import json

f = open("wiki_graph_NYSR_2.json")

d = json.load(f)

In [124]:
print(f"Our json contains a total of {len(d.keys())} entries where each entry is a key value pair.")
print(f"The key is the name of a wikipedia page. The value is a list of pages linked to within that wikipedia page.")

Our json contains a total of 19604 entries where each entry is a key value pair.
The key is the name of a wikipedia page. The value is a list of pages linked to within that wikipedia page.


We can use the networkx library to construct a directed graph from the json. The graph is directed since page links do not imply reverse page links (i.e. just because Pineapple links to 17th century does not mean that 17th century links to pineaple).

In [125]:
import networkx as nx
g = nx.DiGraph(d)

The networkx library allows us to easily compute shortest paths between wikipedia pages using [Dijkstra's Algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm).

In [128]:
source = "New_York_State_Route_373"

# How many links do you need to click to get from NYSR_373 to NYS Legislature?
target = "New_York_State_Legislature"
shortest_path = nx.shortest_path(g, source=source, target=target)
print(f"It takes {len(shortest_path)-1} clicks to go from {source} to {target}")

# How many links do you need to click to get from NYSR_373 to "Enos_T._Throop"?
target = "Enos_T._Throop"
shortest_path = nx.shortest_path(g, source=source, target=target)
print(f"It takes {len(shortest_path)-1} clicks to go from {source} to {target}")

It takes 1 clicks to go from New_York_State_Route_373 to New_York_State_Legislature
It takes 2 clicks to go from New_York_State_Route_373 to Enos_T._Throop


Great! Now we have a way of finding the shortest path between any two wikipedia pages that we scraped. Let's see if we can develop other algorithms for finding the shortest path without access to the graph!

## Step 3: Other Algorithms!

A human being playing the Wikipedia game doesn't have access to the entire graph of Wikipedia pages (if they did, then the game would be a lot easier!) So we wanted to create an algorithm that emulates the behaviour of a human player. When we look for the next link to click, we typically assess how similar each link on a page is to the target page we want to get to. We can do this algorithmically by computing the _semantic similarity_ of each link name with the target page. This yields a greedy algorithm where at each step, we choose the link that is most similar to the target.

In [4]:
# Load the graph

from collections import defaultdict
import json
import numpy as np

with open('wiki_graph.json') as f:
    data = json.load(f)

In [5]:
# Load the language model from Tensorflow
# https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder

import tensorflow_hub as hub

embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

2022-04-17 22:27:05.712890: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-17 22:27:05.712916: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-17 22:27:07.412279: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-17 22:27:07.412308: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-17 22:27:07.412328: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora-ux333): /proc/driver/nvidia/version does not exist
2022-04-17 22:27:07.412490: I tensorflow/core/platform/cpu_featu

In [6]:
def predict(target, links):
    '''
    Given a target, find the link with highest semantic similarity
    '''
    embeddings = embed([target] + links).numpy()
    cosines = np.dot(embeddings[0], embeddings[1:].T)
    return [links[i] for i in cosines.argsort()]

In [11]:
def search(start, end):
    '''
    Apply the semantic similarity greedy search algorithm to find a path from start to end.
    '''
    to_visit = [start]
    visited = defaultdict(bool)
    previous = defaultdict(str)

    while len(to_visit) > 0:
        current = to_visit.pop()

        visited[current] = True
        if current == end:
            break
        if current not in data:
            continue

        links = [l for l in data[current] if not visited[l]]
        links = predict(end, links)
        for l in links:
            to_visit.append(l)
            previous[l] = current

    if visited[end]:
        path = [end]
        current = end
        while previous[current]:
            current = previous[current]
            path.append(current)
        return list(reversed(path))
    else:
        return []

In [12]:
search('New York State Route 373', 'American Antiquarian Society')

['New York State Route 373', 'American Antiquarian Society']

In [13]:
search('New York State Route 373', 'U.S. state')

['New York State Route 373', 'United States', 'U.S. state']

It works for these two examples, so we have a proof of concept!

In [14]:
search('New York State Route 373', 'Henry Louis Gates')

['New York State Route 373',
 'St. Lawrence County, New York',
 'Hopkinton, New York',
 'Adirondack Park',
 'Ausable River (New York)',
 'Keeseville, New York',
 'Chesterfield, New York',
 'Port Kent, New York',
 'Essex County, New York',
 'Albany, New York',
 'Plattsburgh, New York',
 'Lake Champlain Transportation Company',
 'Lake Champlain',
 'New York (state)',
 'New York State Legislature',
 '52nd New York State Legislature',
 'United States',
 'Amtrak',
 'Canadian Pacific Railway',
 'Vermont',
 'Burlington, Vermont',
 'Portland, Maine',
 'Theodore Roosevelt International Highway',
 'Auto trail',
 'Baltimore, Maryland',
 'Portland, Oregon',
 'American Antiquarian Society',
 'Henry Louis Gates']

Only one page links to Henry Louis Gates, which is American Antiquarian Society). Unfortunately American Antiquarian Society is semantically very far from Henry Louis Gates, so it takes a long time for the algorithm to click on American Antiquarian Society, even though it's actually linked to by the starting page.

### Realization: Try it Live!
So far we have been working entirely with a graph that we scraped off wikipedia and stored locally on our machine. This makes the algorithms run quickly since the data are accessible without having to make network requests. However it also means that we are only able to work with a relatively small subgraph of the total wikipedia graph. 

One solution would have been to just scrape more of the wikipedia graph. But this takes a long time as we discuss above.

So instead we decided just to make some minor modifications in order to run our algorithm on the live version of wikipedia!

In [13]:
from WikipediaSearch import WikipediaSearch

source = "Stephen Hawking"  # The page the player is starting at
target = "42 (number)"  # The page where the player is going

player = WikipediaSearch("semantic", verbose=True)
path = player.search(source, target, limit=25)


Stephen Hawking
Age of the universe
Universe
Number
Multicomplex number
P-adic number
Computable number
Superreal number
Super-real number
Split-complex number
Hypercomplex number
Dual-complex number
Bicomplex number
Hyperreal number
Ordinal number
0 (number)
42 (number)


In [14]:
print(f"Player traveled from {source} to {target} in {len(path)} clicks!")

Player traveled from Stephen Hawking to 42 (number) in 17 clicks


Wohoo! In the snippet above, our semantic player was able to get from Stephen Hawking to 42 by requesting Wikipedia directly. We are not passing in a stored graph or any other data structure with wikipedia information. 

This allows us to make all sorts of other explorations that we discuss in the next section!

## Step 4: Competition!
Let's see how our heuristic does against a human and random player! To gather human data, we used [The Wiki Game](https://www.thewikigame.com/group) which allows humans to compete against each other in the Wikipedia Game. The game runs online every 2 minutes and posts the winner and path for every game. Manually we went through and pulled 30 games into a spreadsheet.

Each of the 30 games had a source page and target page. The random and heuristic players played each of the 30 games and their scores were added to the spreadsheet. All of the code and commentary for this can be found in `competition.ipynb`

Let's load in the final competition spreadsheet:

In [9]:
import pandas as pd

df = pd.read_csv("competition_output.csv")
df

,Unnamed: 0,source,target,human_path,semantic_path,random_path,human_path_length,semantic_path_length,random_path_length
0,0,Romance languages,Great Barrier Reef,"['Romance languages', 'Africa', 'Asia', 'Pacif...","['Romance languages', 'Australia', 'Great Barr...",[],6,3.0,NaN
1,1,Natalie Portman,Proto-Indo-European language,"['Natalie Portman', 'United States nationality...",[],[],7,NaN,NaN
2,2,IBM,Human eye,"['IBM', 'Artificial intelligence', 'Facial rec...","['IBM', 'Human resources management', 'Human c...",[],4,24.0,NaN
3,3,Academy Awards,Telecommunications network,"['Academy Awards', 'American Broadcasting Comp...","['Academy Awards', 'Streaming service provider...",[],4,12.0,NaN
4,4,Folk music,John Krasinski,"['Folk music', 'American folk music', 'Music o...",[],[],8,NaN,NaN
5,5,Kindergarten,John Calvin,"['Kindergarten', 'Education in the United Stat...","['Kindergarten', 'Paul Monroe', 'John Dewey', ...",[],5,4.0,NaN
6,6,Glucose,Nintendo Entertainment System,"['Glucose', 'WHO Model List of Essential Medic...","['Glucose', 'Reward system', 'Video game addic...",[],7,5.0,NaN
7,7,Planet,BBC One,"['Planet', 'Galileo Galilei', 'Thomas Harriot'...","['Planet', 'BBC News', 'BBC One']",[],7,3.0,NaN
8,8,Text messaging,Resurrection of Jesus,"['Text messaging', 'Santa Claus', 'Easter Bunn...","['Text messaging', 'Esprit de corps', 'Esprit ...",[],5,8.0,NaN
9,9,Charles Bronson,Role-playing game,"['Charles Bronson', 'Vigilante', 'Duel', '1908...","['Charles Bronson', 'Combat!', 'Combat (disamb...",[],11,8.0,NaN


At a high level, let's see how each player did!

In [15]:
dflens = df[["human_path_length", "semantic_path_length", "random_path_length",]]
dflens.mean(skipna=True)

human_path_length       6.10000
semantic_path_length    8.26087
random_path_length          NaN
dtype: float64

The random player never succeeds at finding the correct path. This is because we did not let it click more than 25 links without calling it a failure.

At first glance the human and semantic player perform rather similarly. However this is not the full picture since the average ignores when the semantic player is unable to get from source to target in 25 clicks

In [22]:
dflens.isna().mean()

human_path_length       0.000000
semantic_path_length    0.233333
random_path_length      1.000000
dtype: float64

The human never fails at reaching the target. The random player always fails. And the semantic player fails 23.3% of the time. 

Note that this is a bit unfair since our human player is actually many human players. We simply selected the winner of each round and used that to represent the human player. Perhaps if we considered the average human player, the semantic player would perform relatively better.

# Future Direction
Despite the semester being over, here are some places that we imagine this project could go if we were to continue it:
* Semantic player with more information. Perhaps the semantic player is given the first paragraph of the article and draws comparisons using this.
* Reinforcement Learning Player. Use the true wikipedia shortest path. Any time that the player moves away from the target, punish. Any time the player moves torwards the target, reward. This raises the question of whether the player would simply learn to memorize the graph. What other information would we need to provide?

[Draft presentation](https://docs.google.com/presentation/d/1GxHRYCPbECe-QAT_uPcoYR31yHpl1f1XA6l_k-nwTvw/edit?usp=sharing)